In [404]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import timeit
import datetime
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nile.api.v1 import clusters
from nile.api.v1 import aggregators as na
from qb2.api.v1 import filters as qf
from nile.api.v1 import filters as nf
from nile.api.v1 import extractors as ne
import ast
from collections import defaultdict
from keras.preprocessing import sequence
import time
from gensim.models import word2vec
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers.advanced_activations import PReLU
import matplotlib
import pickle
import gc
from keras.models import load_model
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from __future__ import print_function
from time import time
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from tqdm import tqdm
import pymorphy2
import nltk

In [405]:
n_samples = 2000
n_features = 20000
n_components = 10
n_top_words = 40
morph = pymorphy2.MorphAnalyzer()
stopWords = stopwords.words('russian')
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=stopWords)#,input="file")

Extracting tf features for LDA...


### Лемматизация и стоп символы

In [406]:
from string import punctuation
from nltk.tokenize import WhitespaceTokenizer
exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
import util_david

In [407]:
lemma_dict = {}
def lemmatizer(word):
    if word in lemma_dict:
        return lemma_dict[word]
    else:
        normal_form = morph.parse(word)[0].normal_form
        lemma_dict[word] = normal_form
        return normal_form

### Загрузка данных  - dataset1.csv =  это переименованный dataset1_ver2.csv

In [408]:
x1  = pd.read_csv('data/dataset1.csv') #
x2  = pd.read_csv('data/dataset2.csv')
x3  = pd.read_csv('data/views.csv',sep = ';')
x4  = pd.read_csv('data/compare_v3.csv',sep = ';')
dist_categories_id = list(x1.CATEGORY_ID.unique())
dist_categories_names = list(x1.CATEGORY_NAME.unique())
mappings = x1[["CATEGORY_ID",'CATEGORY_NAME']].drop_duplicates()
mappings['Number_of_reviews'] = 0

### Отсечка редких категорий

In [409]:
number_of_samples = []
for cat in dist_categories_id:
    number_of_samples.append(x1[x1.CATEGORY_ID==cat].shape[0])
    mappings.Number_of_reviews.values[mappings.CATEGORY_ID.values==cat] = x1[x1.CATEGORY_ID==cat].shape[0]
number_of_samples= np.array(number_of_samples)

In [410]:
dist_categories_id_reduced = list(mappings.CATEGORY_ID.values[mappings.Number_of_reviews>300])
dist_categories_names_reduced = list(mappings.CATEGORY_NAME.values[mappings.Number_of_reviews>300])

In [411]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

### Поработаем со смартфонами в целом

In [441]:
test = x1.TEXT[x1.CATEGORY_ID.values == dist_categories_id_reduced[92]]
y = x1.RATING[x1.CATEGORY_ID.values == dist_categories_id_reduced[92]]

In [442]:
mappings[mappings.CATEGORY_ID == dist_categories_id_reduced[92]]

,CATEGORY_ID,CATEGORY_NAME,Number_of_reviews
160140,2050101,SMARTPHONES,11761


In [444]:
res = []
for person in tqdm(test.values,miniters=10000):
    new_string = ""
    for sentence in person.decode('utf-8').split():
        for char in exclude:
            sentence = sentence.replace(char,"")
        new_string =new_string +' '+ sentence
    new_string_2 = ""
    for word in new_string.split():
        new_string_2 = new_string_2 + " "+ lemmatizer(word)
        

    res.append(new_string_2)
gc.collect()

100%|██████████| 11761/11761 [00:10<00:00, 1107.99it/s]


3639

In [415]:
print ('total unique words' ,len(set(lemma_dict.keys())))
print( 'total unique lemmas ',len(set(lemma_dict.values())))

total unique words 57916
total unique lemmas  29133


### Для смартфонов важные характеристики

In [452]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()

In [447]:
tf = tf_vectorizer.fit_transform(res)
model.fit(tf,y)

In [463]:
fi = model.feature_importances_
tf_feature_names = tf_vectorizer.get_feature_names()
fi = pd.DataFrame(fi)
fi['word'] = tf_feature_names
fi.rename(columns={0:'weight'},inplace=True)
fi.sort(columns='weight',ascending=False,inplace=True)
fi_new = fi[fi.weight>0]
fi_new['cat'] = 0

In [495]:
fi_new.to_excel('feature_weights_mobile.xlsx',index = None)

### Количество важных фичей невелико - почистим вручную - в дальнейшем создадим словарь очистки

In [498]:
fi_adjusted = pd.read_csv('feature_weights_mobile.csv',sep=';',encoding='MacCyrillic')
fi_adjusted = fi_adjusted[fi_adjusted.cat==0]
fi_adjusted.rename(columns={'Unnamed: 3':"Meta_cat"},inplace=True)
for n,i in enumerate(fi_adjusted.weight.values):
    fi_adjusted.weight.values[n] = float(i.replace(',','.'))

In [523]:
fi_adjusted.weight = fi_adjusted.weight/np.sum(fi_adjusted.weight)
fi_adjusted.weight = np.float16(fi_adjusted.weight)
#fi_adjusted_meta = fi_adjusted.groupby('Meta_cat').sum()
fi_adjusted.to_csv('features_for_smartphones.csv',index=None,encoding='MacCyrillic')

In [592]:
fi_adjusted

,weight,word,cat,Meta_cat
6,0.094177,слабый,0,железо
10,0.087646,ремонт,0,гаранти€
15,0.078003,камера,0,камера
26,0.054932,зависать,0,по
38,0.042175,сервис,0,гаранти€
39,0.041107,сырой,0,по
40,0.040619,перезагружатьс€,0,по
42,0.039764,гаранти€,0,гаранти€
45,0.037781,тихий,0,звук
49,0.036072,неудобный,0,дизайн


### Конкретный телефон  - Iphone6

In [595]:
c = 0
iphone_6_review = []
for i in xrange(x1.shape[0]):
    try:
        if 'iPhone 6' in x1.NAME.values[i]:
            c+=1
            iphone_6_review.append(i)
            #break
    except:
        continue
print(c)
iphone6 = x1.ix[iphone_6_review,:]

In [605]:
res_iphone_6 = []
for person in tqdm(iphone6.TEXT.values,miniters=10000):
    new_string = ""
    for sentence in person.decode('utf-8').split():
        for char in exclude:
            sentence = sentence.replace(char,"")
        new_string =new_string +' '+ sentence
    new_string_2 = ""
    for word in new_string.split():
        new_string_2 = new_string_2 + " "+ lemmatizer(word)
        

    res_iphone_6.append(new_string_2)
gc.collect()

100%|██████████| 780/780 [00:01<00:00, 533.39it/s]


131

In [607]:
tf = tf_vectorizer.fit_transform(res_iphone_6)
y = iphone6.RATING
model.fit(tf,y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [608]:
fi_ip = model.feature_importances_
tf_feature_names = tf_vectorizer.get_feature_names()
fi_ip = pd.DataFrame(fi_ip)
fi_ip['word'] = tf_feature_names
fi_ip.rename(columns={0:'weight'},inplace=True)
fi_ip.sort(columns='weight',ascending=False,inplace=True)
fi_ip_new = fi_ip[fi_ip.weight>0]
fi_ip_new['cat'] = 0
fi_ip_new.to_excel('feature_weights_iphone.xlsx',index = None)

In [616]:
fi_ip_adjusted = pd.read_csv('iphone_relevant_features.csv',sep=';',encoding='MacCyrillic')
fi_ip_adjusted = fi_ip_adjusted[fi_ip_adjusted.cat==0]
#fi_ip_adjusted.rename(columns={'Unnamed: 3':"Meta_cat"},inplace=True)
for n,i in enumerate(fi_ip_adjusted.weight.values):
    fi_ip_adjusted.weight.values[n] = float(i.replace(',','.'))

In [617]:
fi_ip_adjusted.weight = fi_ip_adjusted.weight/np.sum(fi_ip_adjusted.weight)
fi_ip_adjusted.weight = np.float16(fi_ip_adjusted.weight)
#fi_adjusted_meta = fi_adjusted.groupby('Meta_cat').sum()
fi_ip_adjusted.to_csv('features_for_iphones.csv',index=None,encoding='MacCyrillic')

In [618]:
fi_ip_adjusted

,weight,word,cat
0,0.156738,тихий,0
11,0.094055,деньга,0
12,0.090454,ремонт,0
14,0.085693,олеофобный,0
29,0.067261,пыль,0
45,0.040833,толща,0
46,0.040833,мерцание,0
47,0.038849,развод,0
54,0.036102,аккуратный,0
61,0.031769,камера,0


### Дальше идут эксперименты с LDA

In [417]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))

for n_topics in [10]:
    lda = LatentDirichletAllocation( max_iter=25,n_topics=n_topics,
                                    learning_method=None,
                                    learning_offset=50.,
                                    random_state=0,n_jobs = 12)
    tf = tf_vectorizer.fit_transform(res)
    t0 = time()
    lda.fit(tf)
    print("done in %0.3fs." % (time() - t0))
    print("\nTopics in LDA model:")
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_top_words)
    for i in xrange(4):
        print(50 * '-')
    

Fitting LDA models with tf features, n_samples=2000 and n_features=20000...
done in 140.032s.

Topics in LDA model:
Topic #0: бюджетник сонька viber пахать полететь экранчик золотистый горилла накрыться проблематично glte охота катушка dual влагозащита пространство замедлить кореец похвалить корея геймер глас граница глазок деревня moto дух ipsматрица xaomi fingerprint фотомодуль просматриваться элджи стереть сойти тонюсенький красивенький шипеть телеграм струя
Topic #1: телефон купить весь год месяц день это работать проблема магазин стать начать неделя покупать время аппарат экран ремонт мочь покупка просто ещё назад брать пользоваться гарантия сказать деньга вообще кнопка пока мвидео сервис взять менять хотеть прийтись отзыв зарядка советовать
Topic #2: это смартфон память который приложение экран карта гб весь камера модель аппарат самый свой мочь работа процессор телефон встроить корпус возможность дать время качество мнение большой ещё xiaomi дисплей новинка плюс устройство разре

In [418]:
cluster = []
for review in tqdm(lda.transform(tf),miniters=10000):
    cluster.append(review.argmax())

100%|██████████| 11761/11761 [00:00<00:00, 929950.22it/s]


In [422]:
test_2 = pd.DataFrame(test)
test_2["Cluster"] = np.array(cluster)
test_2['reviews'] = res

In [423]:
for i in xrange(10):
    print(test_2[test_2.Cluster==i].shape[0],i)

14 0
1220 1
1797 2
9 3
7 4
10 5
4 6
7 7
8687 8
6 9


In [424]:
test_2[test_2.Cluster==1]

,TEXT,Cluster,reviews
160142,"пользуюсь1,5 года начал барахлить сенсор(не ре...",1,пользоваться год начать барахлить сенсорн реа...
160239,После недели использования появился люфт верхн...,1,после неделя использование появиться люфт вер...
160240,"Классный смарт. В эксплуатации 1 год, если бы ...",1,классный смарт в эксплуатация год если бы не ...
160241,Эксплуатирую с ноября 2008 года. Ронял при езд...,1,эксплуатировать с ноябрь год ронять при езда ...
160243,"Отличный телефон, в эксплуатации уже почти 2 г...",1,отличный телефон в эксплуатация уже почти год...
160244,"хороший аппарат, согласен! после него как-то и...",1,хороший аппарат согласный после он както и об...
160251,Купила Nokia e71 в ноябре 2010 года в магазине...,1,купить nokia e в ноябрь год в магазин метр ви...
160398,Этот аппарат без сомнения хорош) У знакомого е...,1,этот аппарат без сомнение хороший у знакомый ...
160502,Подарили такой коммуникатор на 18 лет. Вообще ...,1,подарить такой коммуникатор на год вообще суп...
160516,"Пользуюсь этим телефоном 6 месяцев, предпочита...",1,пользоваться это телефон месяц предпочитать с...


In [386]:
print(test_2.TEXT[test_2.Cluster ==4].values[14])


	Пылесос ужасный, &nbsp;еще и порадовался что сэкономил, но после 15 минут использования понял что бесполезно потратил деньги. Вернулся в магазин , благо там мне пошли навстречу и согласились поменять, поменял на другой фирмы, он стоит в два раза больше, но он того стоит!



In [435]:
res_2 = []
for person in tqdm(test_2[test_2.Cluster==8].TEXT.values,miniters=10000):
    new_string = ""
    for sentence in person.decode('utf-8').split():
        for char in exclude:
            sentence = sentence.replace(char,"")
        new_string =new_string +' '+ sentence
    new_string_2 = ""
    for word in new_string.split():
        new_string_2 = new_string_2 + " "+ lemmatizer(word)
        

    res_2.append(new_string_2)
gc.collect()

100%|██████████| 8687/8687 [00:07<00:00, 1212.11it/s]


0

In [436]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))

for n_topics in [10]:
    lda = LatentDirichletAllocation( max_iter=50,n_topics=n_topics,
                                    learning_method=None,
                                    learning_offset=50.,
                                    random_state=0,n_jobs = 12)
    tf = tf_vectorizer.fit_transform(res_2)
    t0 = time()
    lda.fit(tf)
    print("done in %0.3fs." % (time() - t0))
    print("\nTopics in LDA model:")
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_top_words)
    for i in xrange(4):
        print(100 * '-')
    

Fitting LDA models with tf features, n_samples=2000 and n_features=20000...
done in 191.843s.

Topics in LDA model:
Topic #0: мессенджер освещённость алкатель разбитый видеозвонок ватсап кейс право сэлфи кожа доверять сносный интересно забитый факт нано переносить толковый знак otg показать светиться избавиться удалённый скатить бубен майкрософт господин пятёрка фотоснимок поправить дак ващий волноваться кредит фотографироваться прерываться перемещаться blitz подлагивание
Topic #1: nexus серф сочетаться соответствие внутренность вертекс angry покорить завтра называться проявить выгодно выделяться alcatel birds фоткаюта сколькото округлый внучка звонкий кафе вкупе глобальный переговоры лёгенький размытие залипаний фотовспышка расчитывать подвох сабвеять пойти мировой желтизна зависоновый улавливать синева вотс прятать свидание
Topic #2: мобила посторонний экранчик симпатично увеличение золотистый сбалансировать оплата обед еда родственник xa выложить затвор разблокироваться переписывать

In [437]:
cluster_2 = []
for review in tqdm(lda.transform(tf),miniters=10000):
    cluster_2.append(review.argmax())
test_3 = pd.DataFrame(res_2)
test_3["Cluster"] = np.array(cluster_2)

100%|██████████| 8687/8687 [00:00<00:00, 1038622.58it/s]


In [438]:
for i in xrange(10):
    print(test_3[test_3.Cluster==i].shape[0],i)

1 0
0 1
2 2
0 3
0 4
983 5
1 6
268 7
1 8
7431 9


In [439]:
x1

,Unnamed: 0,PRODUCT,NAME,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,500677,ПО для сервиса М.Видео ЦП ПК пакет Office бесс...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,Я сравнивала цены у разных магазинов на это вс...,28.07.16 10:40:06,5.0,NaN,NaN,NaN,0,0
1,1,500678,ПО для сервиса М.Видео ЦП ПК пакет Office на г...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,"Лучше взять лицензионку, чем комп лечить потом...","Без МС Офиса, как говорится, как без воды - и ...",28.07.16 10:38:03,5.0,NaN,NaN,NaN,0,0
2,2,500679,ПО для сервиса М.Видео ЦП ПК пакет Kaspersky ...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,Совместимость со всеми практически гаджетами. ...,06.11.16 19:51:46,4.0,Универсальность. Быстрота действия. Точность о...,Работает для всех устройств. Но пакет для одно...,1,0,0
3,3,500679,ПО для сервиса М.Видео ЦП ПК пакет Kaspersky ...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,Обеспечивает достойную защиту.,"Универсальная прога, сразу заточена и под Макб...",11.08.16 14:19:12,5.0,NaN,NaN,NaN,0,0
4,4,500787,ПО для сервиса М.Видео ЦП ПК пакет Dr.Web Secu...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,Нормальная штука по доступной цене.,Доктором Вебом пользуюсь уже не первый год - н...,11.08.16 14:23:26,5.0,NaN,NaN,NaN,0,0
5,5,500787,ПО для сервиса М.Видео ЦП ПК пакет Dr.Web Secu...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,Не люблю покупать антивирус сразу на нескольк...,06.11.16 23:50:44,5.0,"Доктор веб люблю больше прочих антивирусов, та...",Все отлично,1,0,0
6,6,500944,ПО для сервиса М.Видео ПК пакет Office для дом...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,Приобрела одновременно с покупкой ноутбука сын...,06.11.16 23:51:49,5.0,Устанавливали самостоятельно. Проблем никаких ...,Минусов не обнаруживается.,1,0,0
7,7,500944,ПО для сервиса М.Видео ПК пакет Office для дом...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,"Срок действия никогда не истекает, не то, что ...",11.08.16 14:24:11,5.0,NaN,NaN,NaN,0,0
8,8,500983,ПО для сервиса М.Видео ПК пакет Office 365 пер...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,"Купил на прошлой неделе для домашнего компа, д...",11.08.16 14:22:33,5.0,NaN,NaN,NaN,0,0
9,9,500983,ПО для сервиса М.Видео ПК пакет Office 365 пер...,2140302,251,PACKAGE SERVICES FOR PC,М.Видео,REVIEW,NaN,"Установилось сразу все как надо, работает нор...",06.11.16 23:49:35,5.0,Современная версия всех приложений Офиса. Все ...,"Нормальный офис, без каких-либо минусов.",1,0,0


### Функция
